### In this code, I use log as ln.

In [1]:
import numpy as np
import math


In [3]:
n = 64
#pas = 0.01

eps = 10e-12
x0 = np.ones([n])
ro = np.array([])
for i in range(n):
    # compute the costs
    tmp = 1- math.exp(-(i+1)*0.02) # note that i takes values from 0 to (n-1) 
    ro = np.append(ro,tmp)
    # compute the probabilities
    x0[i] = math.exp(-(i+1)*0.08) # note that i takes values from 0 to (n-1) 
    
#print(ro)
x0 = x0/np.sum(x0) # fit the probabilities so that sum x0 = 1
#print(x0)
print(np.sum(x0)) 
#print(np.dot(x0,ro))
b = np.dot(x0,ro)
print("b = ", b)
#print(math.log(x0[1]))

0.9999999999999999
b =  0.20464165640900608


In [18]:
def func(x):
    f = np.zeros([n])
    for i in range(n):
        f[i] = x[i]*math.log(x[i]) 
    return np.sum(f)
print("f0 at x0 : ", func(x0))

def gradf(x):
    df = np.zeros([n])
    for i in range(n):
        df[i] = math.log(x[i])+1    
    return df.transpose()
#print("grad f at x0 : ", gradf(x0))

def Hf(x):
    H = np.zeros([n, n])
    for i in range(n):
        #H[i][i] = 1/x[i]
        H[i,i] = 1/x[i]
    return H
#print("Hf: ",Hf(x0))
Hf0 = Hf(x0)
# A is the Affine equality constraint
A = np.ones([2, n])
A[1,:] = ro 
#print("A=",A)
#print(A[0,:].transpose())
def Matrix(Hf,A):
    M = np.zeros([n+2, n+2])
    M[0:n,0:n] = Hf
    #print("Hf before for loop :\n", M) 
    for i in range(n):
        M[i,n]= 1
        M[i,n+1]= ro[i]
        M[n,i]= 1
        M[n+1,i]= ro[i]
    return M
M = Matrix(Hf0,A) # matrix to invert, block matrix
#print("after : \n", M)

f0 at x0 :  -3.489220131238525


# Wolfe condition

In [10]:
def phi(x,t,direction):
    tmp = x + t*direction
    p = func(tmp)
    #x = np.array([x1, x2])
    #tmp = np.dot(x.transpose(),P)
    #f = np.dot(tmp,x)/2-np.dot(q.transpose(),x)
    return p

In [11]:
alpha = 0.2
beta = 0.6
def First_Wolfe(alpha,beta,dphi0,x,t,direction):
    LHS = phi(x,t,direction)
    RHS = phi(x,0,direction) + alpha*dphi0*t
    while (LHS > RHS):
        t = t*beta
        xx = x + t*direction
        LHS = phi(xx,t,direction)
        RHS = phi(x,0,direction) + alpha*dphi0*t
    return t

# Newton's Method for Affine Equality Constraints

In [19]:
Hfk = Hf(x0)
Hfinv = np.linalg.inv(Hfk)
#print(Hf0inv)
#dx = -Hf0inv*(gradf(x0).transpose())
gradfk = gradf(x0)
dx = -np.dot(Hfinv,gradfk)
#print(dx)
criterion = np.dot(np.dot(dx,Hfk),dx)/2
#print("criterion: ",criterion)
xk = x0
i = 0
t = 1

while (criterion > eps):
#for i in range(1):
    M = Matrix(Hfk,A)
    invM = np.linalg.inv(M)
    arr_tmp = np.array([0, 0])
    TT = np.append(gradfk,arr_tmp)
    C =  -np.dot(invM,TT)
    #print(C)
    dxk = C[0:n]
    #print(dxk)
    vk = C[n:n+1]
    #print(vk)
    ## Use Wolfe condition to search for the step (le pas)
    t = 1
    dphi0 =  np.dot(gradfk.transpose(),dxk)
    pas = First_Wolfe(alpha,beta,dphi0,xk,t,dxk)
    print("pas : ", pas)
    xopta = xk
    #print(np.dot(A,dxk))
    #print("b:",np.dot(A,xk))
    #print("sum(xk)",np.sum(xk))
    ##update
    xk = xk + pas*dxk
    gradfk = gradf(xk)
    #print(func(xk))
    Hfk = Hf(xk)
    criterion = np.dot(np.dot(dxk,Hfk),dxk)/2
    i = i +1
    Niter = i
    #print("Niter: ",Niter)
    #xopta = xk
    
print("Niter: ",Niter)
#print("xopt: ",xopta)
fopt = func(xopta)
print('fmin = ', fopt)
print('Entrpoy = fmax = ', -fopt)
print("sum(xopt) = ",np.sum(xopta))
print("<xopta,ro> = ",np.dot(xopta,ro))
print("b = ",b)

pas :  1
pas :  1
pas :  1
pas :  1
pas :  0.36
Niter:  5
fmin =  -3.4999424260009047
Entrpoy = fmax =  3.4999424260009047
sum(xopt) =  1.0
<xopta,ro> =  0.20464165640900595
b =  0.20464165640900608
